### Importing the Libraries

In [6]:
# Needed to import data
import sys
import pandas as pd
import re
import dill
import os 

#os search
import platform

# Time 
import time

# bases
import numpy as np
import scipy

#pre-processing 
from sklearn.preprocessing import StandardScaler

#k-Fold cross 
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

#Ridge 
from sklearn.linear_model import Ridge

#plotting 
import matplotlib.pyplot as plt

#reproducability 
import random
random.seed(42)
np.random.seed(42)

### Setting up paths

In [7]:
sys.path.append('/Users/lakrama/Neuro Project Codes/LSR-Tensor-Ridge-Regression/Data_Sets/HCP')
sys.path.append('/Users/lakrama/Neuro Project Codes/LSR-Tensor-Ridge-Regression/Vector Regression Baseline/Experimental Results/Vector_Base_Line/with k-fold cv kernel mapping before centering')

### Functions Needed


In [8]:
#Row Wise Normalization of Samples
def normalize_rows(matrix: np.ndarray):
    """
    Normalize each row of the given matrix by the norm of the row.
    
    Parameters:
    matrix (numpy.ndarray): The input matrix to be normalized.
    
    Returns:
    numpy.ndarray: The row-normalized matrix.
    """
    # Calculate the L2 norm for each row. Adding a small epsilon to avoid division by zero.
    row_norms = np.linalg.norm(matrix, axis=1, keepdims=True)
    epsilon = 1e-10  # Small value to prevent division by zero
    row_norms[row_norms == 0] = epsilon
    
    # Normalize each row by its norm
    normalized_matrix = matrix / row_norms
    return normalized_matrix


def LRR(X_train,Y_train,X_validatioon,Y_validation,alpha,Y_train_mean = None):
    print(f'X_train inside the LRR:{X_train.shape}')
    #model 
    ridge_regression = Ridge(alpha=alpha,fit_intercept=False)
    #fitting model
    ridge_regression.fit(X_train,Y_train)
    #testing 
    
    if Y_train_mean is not None:
        Y_predicted = ridge_regression.predict(X_validatioon).flatten().reshape(-1,1) + Y_train_mean
    else:
        Y_predicted = ridge_regression.predict(X_validatioon).flatten().reshape(-1,1) 

    #error matrices 

    #nmse
    nmse    = (np.linalg.norm(Y_validation - Y_predicted)**2/(np.linalg.norm(Y_validation)**2))
    #r2
    numerator = np.sum((Y_validation - Y_predicted)**2)
    denominator = np.sum((Y_validation - np.mean(Y_validation))**2)
    r2 = 1 - (numerator / denominator)
    #correlation
    corr    = np.corrcoef(Y_validation.flatten(), Y_predicted.flatten())[0,1]
    
    return nmse,r2,corr



### Hyper Parameteres

In [9]:
#number of splits for K-Fild Cross Validation
n_splits = 10
# Ridge Regression coefficient grid
#the regularization coefficients to search over
alphas = [0,0.00001,0.0001, 0.001, 0.004, 0.007, 0.01, 0.04, 0.07, 0.1, 0.4, 0.7, 1, 1.5, 2, 2.5, 3,3.5, 4, 5, 10, 15, 20]
#[0.07,0.08,0.09,0.1,0.12,0.14,0.16,0.18,2.0,2.5,3.0,3.5,4.0]

#number of seeds
number_of_seeds = 30
n_samples = 550


### Importing Data 

#### 1. Data Loading

In [10]:
#Load fMRI Resting State Data
with open("/Users/lakrama/Neuro Project Codes/LSR-Tensor-Ridge-Regression/Data_Sets/HCP/Resting State FMRI/fmri_rs.npy", "rb") as f:
  fmri_rs = np.load(f)

#Loading Structural FMRI data 
fMRI_language  = (scipy.io.loadmat("/Users/lakrama/Neuro Project Codes/LSR-Tensor-Ridge-Regression/Data_Sets/HCP/Resting State FMRI/features_lang.mat")['features_lang'])

#Take the Transpose so that each Sample is a Row
fmri_rs = fmri_rs.T
fMRI_language = fMRI_language.T

#iterating over seeds



columns = ['Seed','Best_lambda','NMSE','CORR','R2','Time_Taken']
results_df = pd.DataFrame(columns = columns)


for seed in range(number_of_seeds):

    print('')
    print(f'=====================================Seed:{seed}===========================================')

    #Get Split to divide into train + test(loaded data is in the form of features * sampels so need to transpose)
    mat_file = scipy.io.loadmat("/Users/lakrama/Neuro Project Codes/LSR-Tensor-Ridge-Regression/Data_Sets/HCP/Resting State FMRI/MMP_HCP_60_splits.mat")
    seed_1 = mat_file['folds'][f'seed_{seed+1}']
    subject_lists = seed_1[0, 0]['sub_fold'][0, 0]['subject_list']
    test_subjects = [int(item[0]) for item in subject_lists[0,0].flatten()]

    #Get HCP test subjects
    HCP_753_Subjects = []
    with open('/Users/lakrama/Neuro Project Codes/LSR-Tensor-Ridge-Regression/Data_Sets/HCP/Resting State FMRI/MMP_HCP_753_subs.txt', 'r') as file:
        HCP_753_Subjects = [int(re.sub('\n', '', line)) for line in file.readlines()]

    #Put the HCP test subjects into a dataframe
    df = pd.read_csv("/Users/lakrama/Neuro Project Codes/LSR-Tensor-Ridge-Regression/Data_Sets/HCP/Resting State FMRI/MMP_HCP_componentscores.csv")
    df['Subject'] = pd.to_numeric(df['Subject'], errors='coerce')
    df = df[df['Subject'].isin(HCP_753_Subjects)].reset_index(drop = True)

    #Split all our data into a Train and Test Set
    df_train, df_test = df[~df['Subject'].isin(test_subjects)], df[df['Subject'].isin(test_subjects)]

    #Create Train and Test Arrays corresponding to Training and Test Subjects
    train_subjects = df_train.index.to_list()
    test_subjects = df_test.index.to_list()

    #Reshape Labels into Column Vectors

    # Assuming fmri_rs and dMRI_streamlog are numpy arrays or can be converted to numpy arrays
    X_train_fmri = fmri_rs[train_subjects]
    Y_train_fmri = df_train["varimax_cog"].to_numpy().reshape((-1, 1))
    X_test_fmri = fmri_rs[test_subjects]
    Y_test_fmri = df_test["varimax_cog"].to_numpy().reshape((-1, 1))

    X_train_fmri_language  = fMRI_language[train_subjects]
    X_test_fmri_language   = fMRI_language[test_subjects]

    #concatanating the vectors from different feature sets

    # Check if number of samples match
    assert X_train_fmri.shape[0] == X_train_fmri_language.shape[0], "Number of training samples do not match"
    assert X_test_fmri.shape[0] == X_test_fmri_language.shape[0], "Number of testing samples do not match"

    # Concatenate along the features axis (axis=1)
    X_train_combined = np.concatenate((X_train_fmri, X_train_fmri_language), axis=1)
    X_test_combined = np.concatenate((X_test_fmri, X_test_fmri_language), axis=1)

    # The target variables remain the same
    Y_train_combined = Y_train_fmri
    Y_test_combined = Y_test_fmri


    X_train = X_train_combined[:n_samples]
    X_test = X_test_combined
    Y_train = Y_train_combined[:n_samples]
    Y_test = Y_test_combined

        # Verify the shapes
    print("X_train_combined shape:", X_train.shape)
    print("X_test_combined shape:", X_test.shape)
    print("Y_train_combined shape:", Y_train.shape)
    print("Y_test_combined shape:", Y_test.shape)

    #Preprocess Data

    #Projecting in to the kernel space
    X_train = normalize_rows(X_train)
    X_test = normalize_rows(X_test)

    #intercept removal step

    #number of samples in train and test
    n_train = X_train.shape[0]
    n_test = X_test.shape[0]



    # Initialize StandardScaler
    scaler = StandardScaler(with_std = False) #standard scalar only

    # Fit scaler on train data and transform train data
    X_train_scaled = scaler.fit_transform(X_train)

    # Transform test data using the scaler fitted on train data
    X_test_scaled = scaler.transform(X_test)


    #average response value
    Y_train_mean = np.mean(Y_train)

    # Mean centering y_train and y_test
    Y_train = Y_train - Y_train_mean

    #printing the outcomes
    print("Sample mean for each feature (across samples):",scaler.mean_)
    print("Sample variance for each feature (across samples):",scaler.var_)
    print('Response Average:',Y_train_mean)


    # Number of folds we are using 

    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)



    k_fold_results = np.ones(shape=[n_splits,len(alphas),3])*np.inf

    #iterating over the folds
    for fold,(train_ids,validation_ids) in enumerate(kfold.split(X_train)):

        #the training and validatiion data 
        X_train_fold,Y_train_fold = X_train[train_ids],Y_train[train_ids]
        X_validation_fold,Y_validation_fold = X_train[validation_ids],Y_train[validation_ids]
        
        #iterating over the alpha values 
        for alpha_idx,alpha in enumerate(alphas):
            nmse,r2,corr = LRR(X_train_fold,Y_train_fold,X_validation_fold,Y_validation_fold,alpha)
            k_fold_results[fold,alpha_idx,0] = nmse
            k_fold_results[fold,alpha_idx,1] = r2
            k_fold_results[fold,alpha_idx,2] = corr

            print(f"fold = {fold},alpha = {alpha},nmse = {nmse},r2 = {r2}, corr = {corr}")

    #Choosing the best lambda

    #array to hold all the sums
    nmse_sum = np.zeros(len(alphas))

    # Iterate over the folds and alpha values to accumulate nmse values
    for alpha_idx in range(len(alphas)):
        for fold in range(n_splits):
            nmse_sum[alpha_idx] += k_fold_results[fold, alpha_idx, 0]

    #best lambda 

    best_alpha_idx = np.argmin(nmse_sum)
    best_alpha = alphas[best_alpha_idx]        
    print(f"The best alpha value is {best_alpha}")
    average_results = np.mean(k_fold_results, axis=0)
    print(f"The best alpha value is {best_alpha}")
    print(f'average performance metrics: {average_results[best_alpha_idx]}')

    #Choosing the best lambda

    # Iterate over the folds and alpha values to accumulate nmse values
    for alpha_idx in range(len(alphas)):
        for fold in range(n_splits):
            nmse_sum[alpha_idx] += k_fold_results[fold, alpha_idx, 0]

    best_alpha_idx = np.argmin(nmse_sum)
    best_alpha = alphas[best_alpha_idx]        
    average_results = np.mean(k_fold_results, axis=0)
    
    print(f"The best alpha value is {best_alpha}")
    print(f'average performance metrics: {average_results[best_alpha_idx]}')

    #Testing 
    start_time = time.time()
    nmse_best,r2_best,corr_best =  LRR(X_train,Y_train,X_test,Y_test,best_alpha,Y_train_mean=Y_train_mean)
    end_time = time.time()
    elapsed_time = end_time - start_time 
    
    print(f'Time to run one experiment: {elapsed_time}')
    print(f'Test Results: Seed:{seed} Best_Lambda:{best_alpha} NMSE:{nmse_best} CORR:{corr_best} R2:{r2_best}')

    #loading results 
    columns = ['Seed','Best_lambda','NMSE','CORR','R2','Time_Taken']
    
    seed_result_df = pd.DataFrame([{
        'Seed' : seed,
        'Best_lambda' : best_alpha,
        'NMSE' : nmse_best,
        'CORR' : corr_best,
        'R2' :  r2_best,
        'Time_Taken' : elapsed_time 
    }])

    # concatenating results 
    results_df = pd.concat([results_df,seed_result_df],ignore_index = True)

    #saving the results 

    if platform.system() == 'Windows':
        file_path = rf'Enter file path'
    elif platform.system() == 'Darwin':
        file_path = '/Users/lakrama/Neuro Project Codes/LSR-Tensor-Ridge-Regression_All_Data/Vector Regression Baseline/Experimental Results/Vector_Base_Line/with concat k-fold cv kernel map centering fmri/Iterating over seeds_550_samples'

    results_df.to_csv(f'{file_path}/results_{seed}.csv',index = False)


=====================================Seed:0===========================================
X_train_combined shape: (550, 159600)
X_test_combined shape: (76, 159600)
Y_train_combined shape: (550, 1)
Y_test_combined shape: (76, 1)
Sample mean for each feature (across samples): [0.00544843 0.00588438 0.00332152 ... 0.00192157 0.00090728 0.00326901]
Sample variance for each feature (across samples): None
Response Average: 1.1069732851651208
X_train inside the LRR:(495, 159600)
fold = 0,alpha = 0,nmse = 0.4760259114257704,r2 = 0.5239500351502048, corr = 0.7411585401563535
X_train inside the LRR:(495, 159600)
fold = 0,alpha = 1e-05,nmse = 0.47602568517402377,r2 = 0.5239502614133837, corr = 0.741159442763182
X_train inside the LRR:(495, 159600)
fold = 0,alpha = 0.0001,nmse = 0.4760236562364787,r2 = 0.5239522904534504, corr = 0.7411675607674307
X_train inside the LRR:(495, 159600)
fold = 0,alpha = 0.001,nmse = 0.47600408900019525,r2 = 0.5239718586784594, corr = 0.7412482026421103
X_train inside t

/var/folders/r8/5wc8dngx7j55w_gyx0q8q2x80000gn/T/ipykernel_6598/1950958019.py:200: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df,seed_result_df],ignore_index = True)


X_train_combined shape: (550, 159600)
X_test_combined shape: (77, 159600)
Y_train_combined shape: (550, 1)
Y_test_combined shape: (77, 1)
Sample mean for each feature (across samples): [0.00544711 0.00588224 0.00330302 ... 0.00187339 0.00091658 0.00321099]
Sample variance for each feature (across samples): None
Response Average: 1.0917109373968865
X_train inside the LRR:(495, 159600)
fold = 0,alpha = 0,nmse = 0.5429642051412691,r2 = 0.4342414520077277, corr = 0.7200423503909972
X_train inside the LRR:(495, 159600)
fold = 0,alpha = 1e-05,nmse = 0.5429651812713199,r2 = 0.4342404348984691, corr = 0.7200416208006691
X_train inside the LRR:(495, 159600)
fold = 0,alpha = 0.0001,nmse = 0.5429739776311158,r2 = 0.43423126925605915, corr = 0.7200350498673275
X_train inside the LRR:(495, 159600)
fold = 0,alpha = 0.001,nmse = 0.5430630379808717,r2 = 0.4341384700370968, corr = 0.7199688874092739
X_train inside the LRR:(495, 159600)
fold = 0,alpha = 0.004,nmse = 0.5433737496285717,r2 = 0.43381471431